<a href="https://colab.research.google.com/github/ram130849/inception_net/blob/main/InceptionNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Layer
import keras.backend as K
from keras.datasets import cifar10
import matplotlib.pyplot as plt

In [5]:
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D,Dropout,Dense,Input,concatenate,GlobalAveragePooling2D,AveragePooling2D,Flatten
from keras.layers.pooling.max_pooling2d import MaxPooling2D
import cv2
import numpy as np
from keras.datasets import cifar10
from keras import backend as k
from keras.utils import np_utils
import math
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler

In [4]:
no_classes = 10
def load_cifar10_data(rows,cols):
    (x_train,y_train),(x_test,y_test) = cifar10.load_data()
    x_train = np.array([cv2.resize(img,(rows,cols)) for img in x_train[:,:,:,:]])
    x_test = np.array([cv2.resize(img,(rows,cols)) for img in x_test[:,:,:,:]])
    y_train = np_utils.to_categorical(y_train,no_classes)
    y_test = np_utils.to_categorical(y_test,no_classes)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train = x_train/255.0
    x_test = x_test/255.0
    return x_train,y_train,x_test,y_test

In [ ]:
X_train, y_train, X_test, y_test = load_cifar10_data(224, 224)

In [ ]:
def inception(x,filters1,filters3,filters3_reduce,filters5,filters5_reduce,filters_pool_proj,kernel_init,bias_init,name=None):
    conv1 = Conv2D(filters1,(1,1),padding='same',activation='relu',kernel_initializer=kernel_init,bias_initializer=bias_init)
    conv3_r = Conv2D(filters3_reduce,(1,1),padding='same',activation='relu',kernel_initializer=kernel_init,bias_initializer=bias_init)
    conv3 = Conv2D(filters3,(3,3),padding='same',activation='relu',kernel_initializer=kernel_init,bias_initializer=bias_init)
    conv5_r = Conv2D(filters5_reduce,(1,1),padding='same',activation='relu',kernel_initializer=kernel_init,bias_initializer=bias_init)
    conv5 = Conv2D(filters5,(5,5),padding='same',activation='relu',kernel_initializer=kernel_init,bias_initializer=bias_init)
    pool_proj = MaxPool2D((3,3),strides=(1,1),padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj,(1,1),padding='same',activation='relu',kernel_initializer=kernel_init,bias_initializer=bias_init)(x)
    output = concatenate([conv1,conv3,conv3_r,conv5,conv5_r,pool_proj],axis=3,name=name)(pool_proj)
    return output

In [ ]:
kernel_init = keras.initializers.glorot_normal()
bias_init = keras.initializers.Constant(value=0.2)

In [ ]:
input_layer = Input(shape=(224,224,3))
x = Conv2D(64,(7,7),padding='same',strides=(2,2),
           activation='relu',kernel_initializer=kernel_init,
           bias_initializer=bias_init,name='conv1')(input_layer)
x = MaxPool2D((3,3),padding='same',strides=(2,2),
              activation='relu',kernel_initializer=kernel_init,
              bias_initializer=bias_init,name='maxpool1')(x)
x = Conv2D(64,(1,1),padding='same',strides=(1,1),
           activation='relu',kernel_initializer=kernel_init,
           bias_initializer=bias_init,name='conv2a')(x)
x = Conv2D(192,(3,3),padding='same',strides=(2,2),
           activation='relu',kernel_initializer=kernel_init,
           bias_initializer=bias_init,name='conv2b')(x)
x = MaxPool2D((3,3),padding='same',strides=(2,2),activation='relu',name='maxpool2')(x)
x = inception(x,filters1=64,filters3=128,filters3_reduce=96,
              filters5=32,filters5_reduce=16,filters_pool_proj=32,
              kernel_init=kernel_init,bias_init=bias_init,
              name='inception_3a')
x = inception(x,filters1=128,filters3=192,filters3_reduce=128,
              filters5=96,filters5_reduce=32,filters_pool_proj=64,
              kernel_init=kernel_init,bias_init=bias_init,
              name='inception_3b')
x = MaxPool2D((3,3),padding='same',strides=(2,2),name='maxpool_3')(x)
x = inception(x,filters1=192,filters3=208,filters3_reduce=96,
              filters5=48,filters5_reduce=16,filters_pool_proj=64,
              name='inception_4a'
              )
x1 = AveragePooling2D((5,5),strides=3)(x)
x1 = Conv2D(128,(1,1),padding='same',activation='relu')(x1)
x1 = Flatten()(x1)
x1 = Dense(1024,activation='relu')(x1)
x1 = Dropout(0.7)(x1)
x1 = Dense(10,activation='softmax',name='auxillary_output_1')(x1)
x = inception(x,filters1=160,filters3=224,filters3_reduce=112,
              filters5=64,filters5_reduce=24,filters_pool_proj=64,
              name='inception4b')
x = inception(x,filters1=128,filters3=256,filters3_reduce=128,
              filters5=64,filters5_reduce=24,filters_pool_proj=64,
              name='inception4c')
x = inception(x,filters1=112,filters3=288,filters3_reduce=144,
              filters5=64,filters5_reduce=32,filters_pool_proj=64,
              name='inception4d')
x2 = AveragePooling2D((5,5),strides=3)(x)
x2 = Conv2D(128,(1,1),padding='same',activation='relu')(x2)
x2 = Flatten()(x2)
x2 = Dense(1024,activation='relu',kernel_initializer=kernel_init,bias_initializer=bias_init)(x2)
x2 = Dropout(0.7)(x2)
x2 = Dense(10,activation='softmax',kernel_initializer=kernel_init,bias_initializer=bias_init,name='auxillary_output_2')(x2)
x = inception(x,filters1=256,filters3=320,filters3_reduce=160,filters5=128,
                fitlers5_reduce=32,filters_pool_proj=128,name='inception_4e')(x2)
x = MaxPool2D((3,3),padding='same',strides=(2,2),name='maxpool4')(x)
x = inception(x,filters1=256,filters3=320,filters3_reduce=160,filters5=128,
                filters5_reduce=32,filters_pool_proj=128,name='inception_5a')
x = inception(x,filters1=384,filters3=384,filters3_reduce=192,filters5=128,
                filters5_reduce=48,filters_pool_proj=128,name='inception5b')
x = GlobalAveragePooling2D(name='avg_pool5')(x)
x = Dropout(0.4)(x)
x = Dense(10,activation='softmax',name='output')(x)

In [ ]:
model = Model(input_layer,[x,x1,x2],name='inception_v1')
model.summary()

In [1]:
epochs = 100
initial_lrate = 0.01

In [2]:
def decay(epoch, steps=100):
    initial_lrate = 0.01
    drop = 0.96
    epochs_drop = 8
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

In [ ]:
adam = Adam(lr=initial_lrate, momentum=0.9)

lr_sc = LearningRateScheduler(decay, verbose=1)

model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'],
              loss_weights=[1, 0.3, 0.3], optimizer=adam, metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, [y_train, y_train, y_train], validation_data=(X_test, [y_test, y_test, y_test]), epochs=epochs, batch_size=256, callbacks=[lr_sc])

In [6]:
def plot_model_history(train_history,val_history,train_loss_history,val_loss_history):
    plt.plot(train_history, marker='o')
    plt.plot(val_history, marker='o')
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    
    # plot Loss
    plt.plot(train_loss_history, marker='8')
    plt.plot(val_loss_history, marker='8')
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()